# CS 178 Project
## Import part

In [1]:
import os
import numpy as np
import pandas as pd
import cv2

from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Read image for train

In [2]:
def read_pic(data, categories):
    folder_path = 'images/'
    image_paths = data.iloc[:, 1]
    labels = data.iloc[:, 2].str.lower()
    X = []
    label_to_int = {category: i for i, category in enumerate(categories)}
    integer_labels = labels.map(label_to_int)
    y = torch.LongTensor(integer_labels.to_numpy())
    for filename in image_paths:
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (350, 350))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            if img is not None:
                X.append(torch.Tensor(img))
    X = torch.stack(X).unsqueeze(1) # convert to pytorch tensor (batch, channels, w, h)
    return X, y
    
categories1 = [
    "anger", 
    "surprise", 
    "disgust", 
    "fear", 
    "neutral", 
    "happiness", 
    "sadness",
    "contempt"
] # This is for Legend
categories2 = [
    "anger", 
    "surprise", 
    "disgust", 
    "fear", 
    "neutral", 
    "happiness", 
    "sad",
    "contempt"
] # This is for 500_picts

categories3 = [
    "anger", 
    "surprise", 
    "disgust", 
    "fear", 
    "neutral", 
    "happiness", 
    "sad"
]

# Xtr, ytr = read_pic(pd.read_csv('data/legend.csv'), categories1)
# Xva, yva = read_pic(pd.read_csv('data/500_picts_satz.csv'), categories2)
X, y = read_pic(pd.read_csv('data/500_picts_satz.csv'), categories3)
# X, y = read_pic(pd.read_csv('data/legend.csv'), categories1)
# X = X[:1000]
# y = y[:1000]
seed = 1234
Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=seed)

print(Xtr.shape)
print(Xva.shape)
print(ytr)
print(yva)

torch.Size([399, 1, 350, 350])
torch.Size([100, 1, 350, 350])
tensor([5, 6, 5, 0, 4, 0, 4, 5, 0, 0, 5, 5, 3, 4, 5, 6, 5, 5, 4, 5, 3, 3, 5, 0,
        4, 3, 5, 5, 0, 6, 5, 0, 5, 0, 5, 5, 4, 0, 4, 5, 4, 5, 6, 5, 5, 0, 0, 4,
        4, 0, 0, 0, 5, 0, 4, 5, 5, 6, 5, 4, 0, 5, 5, 4, 0, 5, 6, 6, 5, 5, 5, 4,
        3, 5, 4, 0, 5, 5, 5, 5, 3, 5, 0, 4, 6, 3, 5, 4, 0, 5, 4, 5, 0, 0, 5, 4,
        4, 3, 3, 5, 0, 0, 3, 4, 0, 5, 5, 6, 0, 5, 3, 6, 0, 5, 0, 6, 6, 5, 5, 6,
        0, 5, 4, 5, 3, 4, 3, 5, 5, 0, 3, 6, 6, 4, 4, 5, 0, 5, 5, 5, 5, 0, 6, 6,
        5, 5, 4, 5, 0, 0, 0, 3, 5, 0, 0, 5, 6, 5, 6, 5, 0, 5, 5, 4, 0, 5, 0, 3,
        5, 5, 5, 4, 0, 5, 4, 0, 5, 5, 4, 4, 6, 4, 5, 6, 5, 6, 0, 5, 0, 3, 5, 3,
        0, 0, 6, 3, 4, 0, 4, 0, 3, 0, 4, 5, 5, 0, 6, 5, 5, 3, 5, 5, 5, 5, 3, 0,
        4, 5, 5, 0, 5, 5, 5, 0, 6, 0, 4, 0, 5, 0, 6, 0, 3, 4, 0, 5, 3, 5, 0, 5,
        4, 3, 0, 4, 5, 5, 4, 5, 0, 3, 4, 3, 5, 6, 5, 5, 0, 4, 6, 6, 5, 3, 5, 4,
        3, 3, 3, 5, 0, 5, 5, 5, 0, 5, 5, 5, 4, 4, 0, 0, 4,

## CNN Test
### Define a Convolutional Neural Network
To calculate the layer after each convolutional layer, assume $n$ is original image width/height, $k$ is kernel size, $s$ is stripe, and $p$ is padding, we should do $\frac{n + k - 2p}{s} + 1$ to get the new width/height.

Pooling will divide width/height by 2.


In [3]:
class Net(nn.Module):
    def __init__(self, label_size):
        super().__init__()

        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)

        # Convolutional Layer 
        # (in_channels, out_channels, kernel_size) 
        # first in channel is 1 because its greyscale 
        self.conv1 = nn.Conv2d(1, 6, 5) # (6, 350, 350) -> (6, 173, 173)
        self.conv2 = nn.Conv2d(6, 16, 5) # (6, 173, 173) -> (16, 84, 84)

        # Linear Layer
        self.fc1 = nn.Linear(16 * 84 * 84, 120) # shape of image after pooling twice (16, 84, 84)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, label_size)

    def forward(self, x):
        # Conv1 -> Pool -> Conv2 -> Pool -> Lin1 -> Lin2 -> Lin3
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net(7)

In [4]:
for i in range(len(ytr)):
    if ytr[i] < 0:
        print(i)

### Define a Loss function and optimizer

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the network

In [6]:
batch_size = 2
train_dataset = torch.utils.data.TensorDataset(Xtr, ytr)
test_dataset = torch.utils.data.TensorDataset(Xva, yva)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs) 
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 48.013
[1,   200] loss: 1.771
[2,   100] loss: 1.777
[2,   200] loss: 1.566
Finished Training


In [7]:
PATH = './model/cpu_test_net.pth'
torch.save(net.state_dict(), PATH)

### Test model

In [8]:
net = Net(7)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network: {100 * correct // total} %')

Accuracy of the network: 39 %


### Modify CNN

In [10]:
class Net2(nn.Module):
    def __init__(self, conv_layer_channels, fc_layer_channels, kernel_size = 5):
        super().__init__()

        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)

        # Convolutional Layer
        self.conv1 = nn.Conv2d(1, 6, kernel_size) # 1 because its greyscale (in_channels, out_channels, kernel_size) (6, 350, 350) -> (6, 173, 173)
        self.conv2 = nn.Conv2d(6, 16, kernel_size) # (16, 84, 84)

        # Linear Layer
        self.fc1 = nn.Linear(16 * 84 * 84, 120) # shape of image after pooling twice (16, 84, 84)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8) # 8 is label count

    def forward(self, x):
        # Conv1 -> Pool -> Conv2 -> Pool -> Lin1 -> Lin2 -> Lin3
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net2([6, 16], [120, 84])